In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm.notebook
from IPython.display import set_matplotlib_formats
from loaders import load_all_videos, load_thumbnail

tqdm.notebook.tqdm().pandas()

%matplotlib inline
set_matplotlib_formats('svg')

In [2]:
vids = load_all_videos()
vids

,Unnamed: 0,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description,is_GB,is_US
0,0,Jw1Y-zhQURU,17.14.11,John Lewis Christmas Ad 2017 - #MozTheMonster,John Lewis,NaN,2017-11-10T07:38:29.000Z,"christmas|""john lewis christmas""|""john lewis""|...",7224515,55681,10247,9479,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,False,False,False,Click here to continue the story and make your...,True,False
1,1,3s1rvMFUweQ,17.14.11,Taylor Swift: ‚Ä¶Ready for It? (Live) - SNL,Saturday Night Live,NaN,2017-11-12T06:24:44.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",1053632,25561,2294,2757,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,False,False,False,Musical guest Taylor Swift performs ‚Ä¶Ready f...,True,False
2,2,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyonc√©,EminemVEVO,NaN,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyonc√© ...,True,False
3,3,PUTEiSjKwJU,17.14.11,Goals from Salford City vs Class of 92 and Fri...,Salford City Football Club,NaN,2017-11-13T02:30:38.000Z,"Salford City FC|""Salford City""|""Salford""|""Clas...",27833,193,12,37,https://i.ytimg.com/vi/PUTEiSjKwJU/default.jpg,False,False,False,Salford drew 4-4 against the Class of 92 and F...,True,False
4,4,rHwDegptbI4,17.14.11,Dashcam captures truck's near miss with child ...,Cute Girl Videos,NaN,2017-11-13T01:45:13.000Z,[none],9815,30,2,30,https://i.ytimg.com/vi/rHwDegptbI4/default.jpg,False,False,False,Dashcam captures truck's near miss with child ...,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73364,40943,1PhPYr_9zRY,18.14.06,BTS Plays With Puppies While Answering Fan Que...,BuzzFeed Celeb,NaN,2018-05-18T16:39:29.000Z,"BuzzFeed|""BuzzFeedVideo""|""Puppy Interview""|""pu...",8259128,645888,4052,62610,https://i.ytimg.com/vi/1PhPYr_9zRY/default.jpg,False,False,False,"BTS with the PPS, the puppies. These adorable ...",False,True
73365,40945,1h7KV2sjUWY,18.14.06,True Facts : Ant Mutualism,zefrank1,NaN,2018-05-18T01:00:06.000Z,[none],1064798,60008,382,3936,https://i.ytimg.com/vi/1h7KV2sjUWY/default.jpg,False,False,False,NaN,False,True
73366,40946,D6Oy4LfoqsU,18.14.06,I GAVE SAFIYA NYGAARD A PERFECT HAIR MAKEOVER ...,Brad Mondo,NaN,2018-05-18T17:34:22.000Z,I gave safiya nygaard a perfect hair makeover ...,1066451,48068,1032,3992,https://i.ytimg.com/vi/D6Oy4LfoqsU/default.jpg,False,False,False,I had so much fun transforming Safiyas hair in...,False,True
73367,40947,oV0zkMe1K8s,18.14.06,How Black Panther Should Have Ended,How It Should Have Ended,NaN,2018-05-17T17:00:04.000Z,"Black Panther|""HISHE""|""Marvel""|""Infinity War""|...",5660813,192957,2846,13088,https://i.ytimg.com/vi/oV0zkMe1K8s/default.jpg,False,False,False,How Black Panther Should Have EndedWatch More ...,False,True


In [3]:
# create a brand new dataframe for hand-crafted features
features = pd.DataFrame(data={'video_id': vids['video_id'], 'category_id': vids['category_id']})
features

,video_id,category_id
0,Jw1Y-zhQURU,NaN
1,3s1rvMFUweQ,NaN
2,n1WpP7iowLc,NaN
3,PUTEiSjKwJU,NaN
4,rHwDegptbI4,NaN
...,...,...
73364,1PhPYr_9zRY,NaN
73365,1h7KV2sjUWY,NaN
73366,D6Oy4LfoqsU,NaN
73367,oV0zkMe1K8s,NaN


In [4]:
publish_times = pd.to_datetime(vids['publish_time'], utc=True)
features['publish_weekday'] = publish_times.apply(lambda x: x.weekday)
features['publish_hour'] = publish_times.apply(lambda x: x.hour)

In [5]:
likes = vids[vids['ratings_disabled'] == False]['likes']
dislikes = vids[vids['ratings_disabled'] == False]['dislikes']

features['like_ratio'] = likes / (likes + dislikes)
features['likes_per_view'] = likes / vids['views']
features['dislikes_per_view'] = dislikes / vids['views']
features['comments_per_view'] = vids['comment_count'] / vids['views']

In [6]:
import re
from nltk.tokenize.casual import casual_tokenize

def num_chars(text):
    return len(text)

def num_words(text):
    return len(casual_tokenize(text))

def num_uppercase_words(text):
    pattern = r'\b[A-Z]{3,}\b'
    return len(re.findall(pattern, text))

def num_repeated_letters(text):
    pattern = r'\b\w*(\w)\1{2,}\w*\b'
    return len(re.findall(pattern, text))

def num_question_marks(text):
    return text.count('?')

def num_exclamation_marks(text):
    return text.count('!')

def num_periods(text):
    return text.count('.')

def num_numbers(text):
    pattern = '\d+'
    return len(re.findall(pattern, text))

text_features = [
    num_chars, num_words, num_uppercase_words,
    num_question_marks, num_exclamation_marks, num_periods, num_numbers
]

for func in text_features:
    features[f"title_{func.__name__}"] = vids['title'].apply(func)

In [7]:
import string
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import ngrams

stops = set(stopwords.words('english'))
stops.update(string.punctuation)
stemmer = PorterStemmer()


def most_frequent_words(phrases, ngram_size=1, num_most_common=5):
    counter = Counter()
    for phrase in phrases:
        phrase = phrase.lower()
        tokens = casual_tokenize(phrase)
        tokens = [
            stemmer.stem(token)
            for token in tokens
            if token not in stops
        ]
        if ngram_size > 1:
            tokens = ngrams(tokens, ngram_size)
        counter.update(tokens)
    return counter.most_common(num_most_common)

categories = vids['category_id'].dropna().unique()
meaningful_tokens = set()
for ctg in categories:
    titles_in_category = vids[vids['category_id'] == ctg]['title']
    for word, _count in most_frequent_words(titles_in_category):
        if re.match('[a-z0-9]+', word):
            meaningful_tokens.add(word)

titles_tokens = vids['title'].apply(lambda text: set(casual_tokenize(text)))
for token in meaningful_tokens:
    features[f"title__{token}"] = titles_tokens.apply(lambda title: token in title)

In [8]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

features['title_sentiment'] = vids['title'].apply(lambda title: sid.polarity_scores(title)['compound'])

In [9]:
for func in text_features:
    features[f"description_{func.__name__}"] = vids['description'].dropna().apply(func)

In [10]:
features['description_sentiment'] = vids['description'].dropna().apply(lambda desc: sid.polarity_scores(desc)['compound'])

In [11]:
meaningful_tokens = set()
for ctg in categories:
    descriptions_in_category = vids[vids['category_id'] == ctg]['description']
    for word, _count in most_frequent_words(descriptions_in_category.dropna(), 1, 5):
        if re.match('[a-z0-9]+', word) and len(word) > 1 and word != 'video':
            meaningful_tokens.add(word)

descs_tokens = vids['description'].dropna().apply(lambda text: set(casual_tokenize(text)))
for token in meaningful_tokens:
    features[f"description__{token}"] = descs_tokens.apply(lambda desc: token in desc)

In [13]:
def parse_tags(value):
    if value == '[none]':
        return set()
    return set(tag.strip('"').lower() for tag in value.split('|'))

tags = vids['tags'].apply(parse_tags)
meaningful_tags = set()
for ctg in categories:
    tags_counter = Counter()
    for t in tags[vids['category_id'] == ctg]:
        tags_counter.update(t)
    for word, _count in tags_counter.most_common(4):
        meaningful_tags.add(word)

for tag in meaningful_tags:
    features[f"tag__{tag}"] = descs_tokens.apply(lambda tags: tag in tags)

In [36]:
features.to_csv('../youtube_data/features.csv')
features

,video_id,category_id,publish_weekday,publish_hour,like_ratio,likes_per_view,dislikes_per_view,comments_per_view,title_num_chars,title_num_words,...,l0,h1,s1,l1,h2,s2,l2,h3,s3,l3
0,Jw1Y-zhQURU,NaN,4,7,0.844573,0.007707,0.001418,0.001312,45,7,...,0,0,0,0,0,0,0,0,0,0
1,3s1rvMFUweQ,NaN,6,6,0.917645,0.024260,0.002177,0.002617,43,15,...,0,0,0,0,0,0,0,0,0,0
2,n1WpP7iowLc,NaN,4,17,0.947740,0.045891,0.002531,0.007336,43,13,...,0,0,0,0,0,0,0,0,0,0
3,PUTEiSjKwJU,NaN,0,2,0.941463,0.006934,0.000431,0.001329,76,15,...,0,0,0,0,0,0,0,0,0,0
4,rHwDegptbI4,NaN,0,1,0.937500,0.003057,0.000204,0.003057,55,9,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73364,1PhPYr_9zRY,NaN,4,16,0.993766,0.078203,0.000491,0.007581,52,8,...,0,0,0,0,0,0,0,0,0,0
73365,1h7KV2sjUWY,NaN,4,1,0.993674,0.056356,0.000359,0.003696,26,5,...,0,0,0,0,0,0,0,0,0,0
73366,D6Oy4LfoqsU,NaN,4,17,0.978982,0.045073,0.000968,0.003743,84,17,...,0,0,0,0,0,0,0,0,0,0
73367,oV0zkMe1K8s,NaN,3,17,0.985465,0.034086,0.000503,0.002312,35,6,...,0,0,0,0,0,0,0,0,0,0


In [31]:
import PIL
import cv2
import colorgram

def extract_palette(bgr):
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    img = PIL.Image.fromarray(rgb)
    return colorgram.extract(img, 4)

def hsl_attributes(row):
    try:
        im = load_thumbnail(row['video_id'])
        for i, color in enumerate(extract_palette(im)):
            row[f"h{i}"] = int(color.hsl.h / 255 * 360)
            row[f"s{i}"] = int(color.hsl.s / 255 * 100)
            row[f"l{i}"] = int(color.hsl.l / 255 * 100)
    except TypeError:
        pass
    return row

hsl_zeros = { f"{x}{i}": 0 for i in range(4) for x in ['h', 's', 'l'] }
features = features.assign(**hsl_zeros)
features = features.progress_apply(hsl_attributes, axis=1)

KeyboardInterrupt: 